# 2. Creating a simple PDE model - Single Particle Model for a half cell

In the [previous notebook](./1-an-ode-model.ipynb) we showed how to create and solve an ODE model in PyBaMM. In this notebook we show how to create and solve a partial differential equation (PDE) model. PDE models are more complex than ODEs as they include derivatives with respect two different variables (in our problems these will be a spatial variable and a time variable). Additional steps will be required to solve PDE models, like discretising the spatial domain.

As an example, we consider the problem of linear diffusion on a sphere, which can be interpreted as a simplified version for the Single Particle Model for a half cell:

$$
  \frac{\partial c_\mathrm{p}}{\partial t} = \nabla \cdot (D_\mathrm{p} \nabla c_\mathrm{p}),
$$

with the following boundary and initial conditions:

$$
  \left.\frac{\partial c_\mathrm{p}}{\partial r}\right\vert_{r=0} = 0, \quad \left.-D_\mathrm{p}\frac{\partial c_\mathrm{p}}{\partial r}\right\vert_{r=R_\mathrm{p}} = -\frac{i(t)}{a_\mathrm{p} F L_\mathrm{p}}, \quad \left.c_\mathrm{p}\right\vert_{t=0} = c_{\mathrm{p}0},
$$

and the following definition of the cell voltage:
$$
\begin{aligned}
V(t) &= U_\mathrm{p}(c_\mathrm{p}) - \frac{2RT}{F} \sinh^{-1} \left( \frac{i(t)}{a_\mathrm{p} L_\mathrm{p} j_{\mathrm{p}0}} \right),\\
j_{\mathrm{p}0} &= F k_\mathrm{p} \left.\sqrt{c_\mathrm{p} \left(c_\mathrm{p}^{\max} - c_\mathrm{p}\right)}\right\vert_{r=R_\mathrm{p}}.
\end{aligned}
$$

Here the model variable is the positive particle concentration $c_\mathrm{p}$ and the voltage $V(t)$ is the output variable, where $j_{\mathrm{p}0}$ is the exchange current density. The model has the following parameters: the lithium diffusivity $D_\mathrm{p}$, the particle radius $R_\mathrm{p}$, the particle surface area density $a_\mathrm{p}$, the electrode thickness $L_\mathrm{p}$, the initial concentration $c_{\mathrm{p}0}$, the temperature $T$, the reaction rate $k_\mathrm{p}$, and the maximum particle concentration $c_\mathrm{p}^{\max}$. The model also has the open-circuit potential $U_\mathrm{p}$ and applied current density $i(t)$ (i.e. the applied current divided by the current collector area) as function parameters, and the Faraday constant $F$ and the universal gas constant $R$. We have used subscript $\mathrm{p}$ throughout the model to denote the positive particle/electrode, even though in this case we only consider a half cell, to be consistent with the full cell notation we will use in future notebooks.

As before, we begin by importing the PyBaMM library into this notebook:

In [1]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm


[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Setting up the model

As in the previous example, we start with a [`pybamm.BaseModel`](https://docs.pybamm.org/en/stable/source/api/models/base_models/base_model.html#pybamm.BaseModel) object and define our model variables. Since we are now solving a PDE we need to tell PyBaMM the domain each variable belongs to so that it can be discretised in space in the correct way. This is done by passing the keyword argument `domain`, and in this example we choose the domain "negative particle". This argument is a string and we will later on define the geometry of the domain.

In [2]:
model = pybamm.BaseModel()

c_p = pybamm.Variable("Positive particle concentration [mol.m-3]", domain="positive particle")

Note that we have given our variable the (useful) name "Positive particle concentration", but the symbol representing this variable is simply `c_p`. As noted in the previous section, the name of the variable is arbitrary and you can choose any variable name that is most useful/meaningful to you but for the notebook we will stick to the PyBaMM convention.

Next we define the parameters involved in the model:

In [3]:
# constant parameters
D_p = pybamm.Parameter("Positive electrode diffusivity [m2.s-1]")
R_p = pybamm.Parameter("Positive particle radius [m]")
epsilon_s_p = pybamm.Parameter("Positive electrode active material volume fraction")
a_p = 3 * epsilon_s_p / R_p # assume spherical particles
L_p = pybamm.Parameter("Positive electrode thickness [m]")
T = pybamm.Parameter("Temperature [K]")
c_p_0 = pybamm.Parameter("Initial concentration in positive electrode [mol.m-3]")
c_p_max = pybamm.Parameter("Maximum concentration in positive electrode [mol.m-3]")
k_p = pybamm.Parameter("Positive electrode reaction rate [m.s-1]")

# function parameters
i = pybamm.FunctionParameter("Current density [A.m-2]", {"Time [s]": pybamm.t})
U_p = pybamm.FunctionParameter("Positive electrode OCV [V]", {"x_p": c_p / c_p_max})

We do not need to define universal constants as parameters because PyBaMM has them built-in already:

In [4]:
R = pybamm.constants.R
F = pybamm.constants.F

We then state out governing equations. Sometime it is useful to define intermediate quantities in order to express the governing equations more easily. In this example we define the flux, then define the rhs to be minus the divergence of the flux. The equation is then added to the dictionary `model.rhs`

In [5]:
N = - D_p * pybamm.grad(c_p)  # define the flux
dcpdt = -pybamm.div(N)  # define the rhs equation

model.rhs = {c_p: dcpdt}  # add the equation to rhs dictionary

Unlike ODE models, PDE models require both initial and boundary conditions. Similar to initial conditions, boundary conditions can be added using the dictionary [`model.boundary_conditions`](https://docs.pybamm.org/en/stable/source/api/models/base_models/base_model.html#pybamm.BaseModel.boundary_conditions). Boundary conditions for each variable are provided as a dictionary of the form `{side: (value, type)}`, where, in 1D, side can be "left" or "right", value is the value of the boundary conditions, and type is the type of boundary condition (at present, this can be ["Dirichlet"](https://en.wikipedia.org/wiki/Dirichlet_boundary_condition) or ["Neumann"](https://en.wikipedia.org/wiki/Neumann_boundary_condition)).

In [6]:
# initial conditions
model.initial_conditions = {c_p: c_p_0}

# boundary conditions
lbc = pybamm.Scalar(0)
rbc = i / (a_p * F * L_p * D_p)
model.boundary_conditions = {c_p: {"left": (lbc, "Neumann"), "right": (rbc, "Neumann")}}

Finally, we add all the variables of interest to the [`model.variables`](https://docs.pybamm.org/en/stable/source/api/models/base_models/base_model.html#pybamm.BaseModel.variables) dictionary. In addition the concentration and voltage, we add the flux and the surface concentration. The surface value of a variable can be obtained calling the `pybamm.surf` method.

In [7]:
# define auxiliary expressions
c_p_surf = pybamm.surf(c_p)
j_p_0 = F * k_p * (c_p_surf * (c_p_max - c_p_surf)) ** 0.5
V = pybamm.surf(U_p) - 2 * R * T / F * pybamm.arcsinh(i / (a_p * L_p * j_p_0))

# define the model variables
model.variables = {
    "Positive particle concentration [mol.m-3]": c_p,
    "Positive particle surface concentration [mol.m-3]": c_p_surf,
    "Voltage [V]": V,
}

## Using the model

Now the model is now completely defined, we can proceed to solve it. In the previous notebook, we had an ODE model so we could proceed to solve it directly. But now we have a PDE model, so we need to take some additional steps. To handle PDEs, PyBaMM uses the [method of lines](http://www.scholarpedia.org/article/Method_of_lines). In a nutshell, this method consists in replacing the continuous spatial operators with a discrete version (a step known as discretisation), which yields the discretised model. Then, the discretised model is integrated in time as we did with the ODE model.

Discretising a PDE means transforming it from a continuous form, which involves variables that can change smoothly, into a discrete form, which involves variables that change in distinct steps or intervals. This process converts the continuous problem, which is typically defined over a continuous domain, into a set of algebraic equations that can be solved using numerical methods. Essentially, discretisation breaks down the PDE into a finite number of simpler equations that approximate the solution at specific points in the domain. This makes it possible to find the solution at these set points using computational techniques.

To discretise a PDE we need to define a mesh and a spatial method. The mesh defines the number and position of points at which we want to compute the solution. The spatial method defines how we approximate the spatial operators (e.g. divergence, gradients...) with a discrete counterpart.

### Defining a geometry
But before discretising, we need to specify the geometry, as the model above is geometry agnostic. First, we need to define the spatial variables in a similar way to how we defined the other model variables. The relevant class is [`pybamm.SpatialVariable`](https://docs.pybamm.org/en/stable/source/api/expression_tree/independent_variable.html#pybamm.SpatialVariable):

In [8]:
r = pybamm.SpatialVariable(
    "r", domain=["positive particle"], coord_sys="spherical polar"
)

As with the positive particle concentration variable above, we give the spatial variable an informative name (`"r"`) but the variable is represented by `r` in the code (in this case it is a bit more confusing as the names are more similar). The domain needs to match the domain we have defined in our concentration variable, and the coordinate system can be chosen from `"cartesian"`, `"cylindrical polar"` and `"spherical polar"`.

Now we can define the geometry on which we wish to solve the model using a nested dictionary. The first key is the domain name (here "negative particle") and the
entry is a dictionary giving the limits of the domain.

In [9]:
geometry = {"positive particle": {r: {"min": pybamm.Scalar(0), "max": R_p}}}

### Defining a mesh

As we explained earlier, to discretise our model we first need to define a mesh. In this case we want a uniform one-dimensional mesh with 20 points. This means the solution will be computed at 20 points along the particle radius, all evenly spaced. To define the mesh, we use the [`pybamm.Mesh`](https://docs.pybamm.org/en/stable/source/api/meshes/meshes.html#pybamm.Mesh)
class. As well as the geometry, the mesh class also takes a dictionary of submesh types (see below, for more details), and a dictionary specifying the number of mesh points for each spatial variable.

In [10]:
submesh_types = {"positive particle": pybamm.Uniform1DSubMesh}
var_pts = {r: 20}

Here we have used the [`pybamm.Uniform1DSubMesh`](https://docs.pybamm.org/en/stable/source/api/meshes/one_dimensional_submeshes.html#pybamm.Uniform1DSubMesh) class to create a uniform mesh.
This class does not require any parameters, and so we can pass it directly to the `submesh_types` dictionary. However, many other submesh types can take additional parameters.  Example of meshes that do require parameters include the [`pybamm.Exponential1DSubMesh`](https://docs.pybamm.org/en/stable/source/api/meshes/one_dimensional_submeshes.html#pybamm.Exponential1DSubMesh) which clusters points close to one or both
boundaries using an exponential rule. It takes a parameter which sets how closely the points are clustered together, and also lets the users select the side on which more points should be clustered. For more information on meshes, please see the [documentation](https://docs.pybamm.org/en/stable/source/api/meshes/index.html).

### Defining the spatial methods

The second component we need to discretise our model is the spatial method to use. At the moment PyBaMM has the [`pybamm.FiniteVolume`](https://docs.pybamm.org/en/latest/source/api/spatial_methods/finite_volume.html) and the [`pybamm.SpectralVolume`](https://docs.pybamm.org/en/latest/source/api/spatial_methods/spectral_volume.html) methods available. The spatial methods need to be passed as a dictionary, with the keys being the various domains and the values the method to use in each domain. In this case we only have one domain, so we simply have:

In [11]:
spatial_methods = {"positive particle": pybamm.FiniteVolume()}

### Solving the model

Now we can proceed to solve the model. We first need to define the parameter values.

In [12]:
import numpy as np

def nmc_LGM50_ocp_Chen2020(sto):
    u_eq = (
        -0.8090 * sto
        + 4.4875
        - 0.0428 * np.tanh(18.5138 * (sto - 0.5542))
        - 17.7326 * np.tanh(15.7890 * (sto - 0.3117))
        + 17.5842 * np.tanh(15.9308 * (sto - 0.3120))
    )

    return u_eq


parameter_values = pybamm.ParameterValues(
    {
        "Positive electrode diffusivity [m2.s-1]": 4e-15,
        "Positive particle radius [m]": 5.22e-06,
        "Positive electrode active material volume fraction": 0.665,
        "Positive electrode thickness [m]": 7.56e-05,
        "Temperature [K]": 298.15,
        "Initial concentration in positive electrode [mol.m-3]": 17038.0,
        "Maximum concentration in positive electrode [mol.m-3]": 63104.0,
        "Positive electrode reaction rate [m.s-1]": 1.1e-9,
        "Current density [A.m-2]": 50,
        "Positive electrode OCV [V]": nmc_LGM50_ocp_Chen2020,
    }
)

With the parameter values, we now have all the elements we need to solve the model. We can still use the simulation class, but in addition to the model and parameter values, we need to pass the `geometry`, `submesh_types`, `var_pts` and `spatial_methods` as additional keyword variables. Then we can just solve the simulation and plot the results as usual:

In [13]:
sim = pybamm.Simulation(
    model,
    parameter_values=parameter_values,
    geometry=geometry,
    submesh_types=submesh_types,
    var_pts=var_pts,
    spatial_methods=spatial_methods,
)

sim.solve([0, 3600])

sim.plot([
    "Positive particle concentration [mol.m-3]",
    "Positive particle surface concentration [mol.m-3]",
    "Voltage [V]"
])

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

In this tutorial we have shown how to build and solve a PDE model in PyBaMM, for which we introduced the concept of discretising PDEs. In the next tutorial we will show how to wrap a model into a class, which is a lot more compact and makes it easier to share the models with other people.

## References

The relevant papers for this notebook are:

In [14]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[3] Valentin Sulzer, Scott G. Marquis, Robert Timms, Martin Robinson, and S. Jon Chapman. Python Battery Mathematical Modelling (PyBaMM). Journal of Open Research Software, 9(1):14, 2021. doi:10.5334/jors.309.

